# **Spit some [tensor] flow**

We need to learn the intricacies of tensorflow to master deep learning



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
print(tf.__version__)

2.2.0


In [2]:
!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

--2020-06-08 19:59:26--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.1’

ml-1m.zip.1         100%[===================>]   5.64M  4.15MB/s    in 1.4s    

2020-06-08 19:59:28 (4.15 MB/s) - ‘ml-1m.zip.1’ saved [5917549/5917549]



In [3]:
!ls

ml-1m  ml-1m.zip  ml-1m.zip.1  sample_data


In [4]:
!unzip ml-1m.zip

Archive:  ml-1m.zip
replace ml-1m/movies.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [5]:
!ls

ml-1m  ml-1m.zip  ml-1m.zip.1  sample_data


In [0]:
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [12]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip

--2020-06-08 20:08:02--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  3.46MB/s    in 1.4s    

2020-06-08 20:08:04 (3.46 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]



In [13]:
!unzip ml-100k.zip

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


In [0]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')


In [35]:
training_set.shape

(79999, 4)

In [34]:
test_set.shape

(19999, 4)

In [0]:
num_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
num_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [0]:
def transform(data):
    transformed = []
    for user_id in range(1, num_users + 1):
        id_movies = data[:,1][data[:,0] == user_id]
        id_ratings = data[:,2][data[:,0] == user_id]
        ratings = np.zeros(num_movies)
        ratings[id_movies - 1] = id_ratings
        transformed.append(list(ratings))
    return transformed


In [0]:
training_set = transform(training_set)
test_set = transform(test_set)

In [39]:
len(training_set)

943

In [40]:
len(training_set[0])

1682

In [41]:
num_movies

1682

In [0]:
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model

In [0]:
X_train = np.array(training_set)
X_test = np.array(test_set)

In [44]:
print(X_train.shape)
print(X_test.shape)

(943, 1682)
(943, 1682)


In [45]:
num_users

943

In [46]:
X_train[0].shape

(1682,)

In [50]:
set(X_train[0])

{0.0, 1.0, 2.0, 3.0, 4.0, 5.0}

In [0]:
input_shape = X_train[0].shape

In [0]:
i_layer = Input(shape = input_shape)
h_layer = Dense(512, activation='relu')(i_layer)
h_layer = Dense(254, activation='relu')(h_layer)
h_layer = Dense(128, activation='relu')(h_layer)
h_layer = Dense(254, activation='relu')(h_layer)
h_layer = Dense(512, activation='relu')(h_layer)
o_layer = Dense(X_train[0].shape[0], activation=None)(h_layer)
model = Model(i_layer, o_layer)


In [62]:
model.compile(optimizer='adam', 
              loss = "mse")

report = model.fit(X_train, X_train, epochs=100, batch_size=10)

Epoch 1/100
95/95 [==============================] - 1s 13ms/step - loss: 0.5212
Epoch 2/100
95/95 [==============================] - 1s 13ms/step - loss: 0.4747
Epoch 3/100
95/95 [==============================] - 1s 13ms/step - loss: 0.4580
Epoch 4/100
95/95 [==============================] - 1s 13ms/step - loss: 0.4453
Epoch 5/100
95/95 [==============================] - 1s 13ms/step - loss: 0.4387
Epoch 6/100
95/95 [==============================] - 1s 13ms/step - loss: 0.4293
Epoch 7/100
95/95 [==============================] - 1s 13ms/step - loss: 0.4209
Epoch 8/100
95/95 [==============================] - 1s 13ms/step - loss: 0.4187
Epoch 9/100
95/95 [==============================] - 1s 13ms/step - loss: 0.4172
Epoch 10/100
95/95 [==============================] - 1s 13ms/step - loss: 0.4176
Epoch 11/100
95/95 [==============================] - 1s 13ms/step - loss: 0.4081
Epoch 12/100
95/95 [==============================] - 1s 13ms/step - loss: 0.4001
Epoch 13/100
95/95 [=====

In [0]:
X_decoded = model.predict(X_test)

In [93]:
X_decoded.shape

(943, 1682)

In [94]:
X_test.shape

(943, 1682)

In [0]:
def map(_temp):
  for i in range(len(_temp)):
    interval = (_temp.max()-_temp.min())/5
    if _temp[i]>=_temp.min() and _temp[i]<(_temp.min()+(1* interval)):
      _temp[i] = 1
    elif (_temp[i]>=(_temp.min()+(1* interval)) and (_temp[i]<(_temp.min()+(2* interval)))):
      _temp[i] = 2
    elif (_temp[i]>=(_temp.min()+(2* interval)) and (_temp[i]<(_temp.min()+(3* interval)))):
      _temp[i] = 3
    elif (_temp[i]>=(_temp.min()+(3* interval)) and (_temp[i]<(_temp.min()+(4* interval)))):
      _temp[i] = 4
    elif (_temp[i]>=(_temp.min()+(4* interval)) and (_temp[i]<(_temp.min()+(5* interval)))):
      _temp[i] = 5
    else:
      pass
  return _temp

In [97]:
idx = np.random.randint(0, len(X_test))
for i in range(len(X_test[idx])):
  if X_test[idx][i] != 0:
    print("Rating by User {u} for Movie {a} = {b}".format(u = idx, a=i, b=X_test[idx][i]))
    print("Prediction by User {u} for Movie {a} = {b}".format(u = idx, a=i, b=map(X_decoded[idx])[i]))

Rating by User 389 for Movie 99 = 5.0
Prediction by User 389 for Movie 99 = 1.0
Rating by User 389 for Movie 123 = 4.0
Prediction by User 389 for Movie 123 = 1.0
Rating by User 389 for Movie 180 = 4.0
Prediction by User 389 for Movie 180 = 4.0
Rating by User 389 for Movie 288 = 3.0
Prediction by User 389 for Movie 288 = 4.0
Rating by User 389 for Movie 303 = 5.0
Prediction by User 389 for Movie 303 = 4.0
Rating by User 389 for Movie 327 = 4.0
Prediction by User 389 for Movie 327 = 4.0
Rating by User 389 for Movie 330 = 2.0
Prediction by User 389 for Movie 330 = 2.0
Rating by User 389 for Movie 712 = 4.0
Prediction by User 389 for Movie 712 = 1.0
Rating by User 389 for Movie 739 = 4.0
Prediction by User 389 for Movie 739 = 1.0
Rating by User 389 for Movie 988 = 5.0
Prediction by User 389 for Movie 988 = 1.0
Rating by User 389 for Movie 989 = 4.0
Prediction by User 389 for Movie 989 = 2.0
